# Imports

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from models import LeNet5Variant
from trainer import MyTrainer

# Data prep

# Loads the data

In [12]:
train_path = "./data/raw/train.csv"
test_path = "./data/raw/test.csv"

In [13]:
df_train_main = pd.read_csv(train_path)
df_train, df_dev = train_test_split(df_train_main, test_size=0.2)
df_path = pd.read_csv(test_path)